In [1]:
import ibm_db

dsn_hostname = "815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "ylg89839"        # e.g. "abc12345"
dsn_pwd = "juFr8vgUo7ILSTyg"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "30367"                # e.g. "32733"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"



#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)
print("")

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=30367;PROTOCOL=TCPIP;UID=ylg89839;PWD=juFr8vgUo7ILSTyg;SECURITY=SSL;



In [2]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )
print("")


Connected to database:  BLUDB as user:  ylg89839 on host:  815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud



In [17]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [18]:
#4
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, " \
              "fname VARCHAR(20) NOT NULL, " \
              "LNAME VARCHAR(20) NOT NULL, " \
              "CITY VARCHAR(20) NOT NULL, " \
              "CCODE CHAR(2) NOT NULL)"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)


In [19]:
#5 insert values
insertQuery = "insert into INSTRUCTOR values " \
              "(1, 'Rav', 'Ahuja', 'TORONTO', 'CA')," \
              "(2, 'Raul', 'Chong', 'Markham', 'CA'), " \
              "(3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [20]:
#6 Query data in the table
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

In [21]:
#Fetch the Dictionary of first row
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [22]:
#print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))


 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [23]:
#7
updateStmt = ibm_db.exec_immediate(conn, "UPDATE INSTRUCTOR SET CITY = 'MOOSETOWN' WHERE FNAME='Rav'")


In [24]:
import pandas as pd
import ibm_db_dbi
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

#query statement to retrieve all rows in INSTRUCTOR table
#retrieve the query results into a pandas dataframe
df = pd.read_sql("select * from INSTRUCTOR", pconn)

#print just the LNAME for first row in the pandas data frame
df.LNAME[0]


'Ahuja'

In [25]:
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [26]:
df.shape

(3, 5)

In [27]:
#retrieve stats of all attributes
df.describe(include="all")
# df["FNAME].describe if for one column (attribute) only


,ID,FNAME,LNAME,CITY,CCODE
count,3.0,3,3,3,3
unique,NaN,3,3,3,2
top,NaN,Rav,Ahuja,MOOSETOWN,CA
freq,NaN,1,1,1,2
mean,2.0,NaN,NaN,NaN,NaN
std,1.0,NaN,NaN,NaN,NaN
min,1.0,NaN,NaN,NaN,NaN
25%,1.5,NaN,NaN,NaN,NaN
50%,2.0,NaN,NaN,NaN,NaN
75%,2.5,NaN,NaN,NaN,NaN


In [28]:
#close connection
ibm_db.close(conn)

True